In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import ipywidgets as widgets
from IPython.display import clear_output

In [2]:
def compras_grafs(fii, data_ini, data_fim):
    
# ------------- SUBSET -------------

    # Monta filtro e verifica se é válido
    filtro = (df['Ativo']==fii) & (df['Data']>=data_ini) & (df['Data']<=data_fim) & (df['Operação'].isin(['C','SE']))
    if df[filtro].empty:
        print(f'Não há operações de compra de {fii} neste período.')
        return

    # Filtra DataFrame original
    df_filtrado = df[filtro].copy()

# ------------- CORES -------------

    color_back = 'Gainsboro'
    color_graf = 'firebrick'
    color_grid = 'DarkGray'

# ------------- CANVAS -------------

    # Inicia figura e define tamanho
    fig = plt.figure(figsize=(26, 13))
    # Título
    fig.suptitle(fii, fontsize=20, weight='bold')
    # Cria subplots
    ax1 = plt.subplot(221)  #121: 2 Linhas, 2 Colunas, Subplot 1
    ax2 = plt.subplot(222)
    ax3 = plt.subplot(223)
    #ax4 = plt.subplot(224)
    fig.subplots_adjust(top=0.93) #Afastamento dos subplots do topo da figura
                                 #Permite o ajuste fino do espaço entre o título e os subplots 
    
# ------------- GRÁFICO COTAS POR FAIXA -------------
    
    # Cria lista de valores por faixa
    df_val = list(map(lambda i: int(i), df_filtrado['Valor']))

    # Define escala auto-ajustável no eixo X
    mi = min(df_val)
    ma = max(df_val)
    barras = ma - mi + 1
    max_bars = 15
    passo_x = (barras // max_bars) + bool(barras % max_bars)  #Soma 1 se houver resto da divisão.
    eixo_x = np.arange(mi, ma+1+passo_x, passo_x)  #Soma 1 no max por causa do "exclusive".
    
    # Configura gráfico
    ax2.hist(df_val, weights=df_filtrado['Quantidade'], rwidth=0.7, bins=eixo_x, color=color_graf, alpha=0.9)
    ax2.rwidth=0.7
    ax2.set_facecolor(color_back)
    ax2.set_axisbelow(True)
    ax2.grid(axis='y', color=color_grid)
    ax2.tick_params(labelright=True)
    ax2.set_ylabel('Cotas', fontsize=14, weight='bold', labelpad=10)
    ax2.set_xlabel('Valor', fontsize=14, weight='bold', labelpad=10)

    # Configura ticks do eixo X
    ax2.set_xticks(eixo_x)
    
    # Define escala auto-ajustável no eixo Y e configura ticks
    tick = int(max(ax2.get_yticks()))  #Lê maior y_tick
    max_yticks = 20
    passo_y = (tick // max_yticks) + bool(tick % max_yticks)
    eixo_y = np.arange(0, tick+1, passo_y)  #Configura a faixa em inteiros
    ax2.set_yticks(eixo_y)

# ------------- GRÁFICO SCATTER -------------
    
    # Dados
    x = df_filtrado['Data']
    y = df_filtrado['Valor']
    markers = list(map(lambda i: i*80, df_filtrado['Quantidade']))

    # Configura gráfico
    ax1.scatter(x, y, s=markers, color=color_graf, edgecolors='white', linewidth=1.2, alpha=0.75)
    ax1.set_facecolor(color_back)
    ax1.set_axisbelow(True)
    ax1.grid(color=color_grid)
    ax1.tick_params(labelright=True)
    ax1.set_ylabel('Valor', fontsize=14, weight='bold', labelpad=10)
    
    # Escala do eixo X (o automático expande demais)
    folga = 2 
    data_min = min(df_filtrado['Data']) - timedelta(days=folga)
    data_max = max(df_filtrado['Data']) + timedelta(days=folga)
    ax1.set_xlim(data_min, data_max)

# ------------- GRÁFICO PREÇO MÉDIO -------------    

    # Filtro do Dataframe
    filtro = (df['Ativo']==fii) & (df['Data']>=data_ini) & (df['Data']<=data_fim) & (df['Operação'].isin(['C','V','SE']))

    # Configura gráfico
    ax3.plot(df[filtro]['Data'], df[filtro]['PM'], color=color_graf, alpha=0.8, marker='o', markersize=9, markeredgecolor='white', linestyle='dotted', linewidth=2)
    ax3.set_facecolor(color_back)
    ax3.set_axisbelow(True)
    ax3.grid(color=color_grid)
    ax3.tick_params(labelright=True)
    ax3.set_ylabel('Preço Médio', fontsize=14, weight='bold', labelpad=10)
    ax3.set_xlim(data_min, data_max)

# ------------- MOSTRAR -------------
    
    # Mostra gráfico
    plt.show()

In [3]:
def compras_clicked(b):
    with output:
        # Limpa gráfico anterior antes de mostrar o novo, esperando o novo aparecer
        clear_output(wait=True)
        
        # Lê dados dos widgets
        fii = fii_dropdown.value
        data_ini = datetime.combine(data_ini_picker.value, datetime.min.time()) # Data vem sem "time" do widget
        data_fim = datetime.combine(data_fim_picker.value, datetime.min.time()) # Data vem sem "time" do widget
        
        # Verifica se data inicial não é maior que final
        if data_ini > data_fim:
            print('ERRO: Data inicial maior que a final!!!')
        # Chama função pra mostrar gráfico
        else:
            compras_grafs(fii, data_ini, data_fim)

In [4]:
def atualiza_lista_fiis(ck):
    lista_fiis = sorted(df[df['Ativo'].str.endswith('11')]['Ativo'].unique())
    if ck:
        lista_fiis = df[(df['Ativo'].str.endswith('11')) & (df['Operação'].isin(['C','V','SE']))].copy()
        lista_fiis = lista_fiis.groupby('Ativo')['Quantidade'].sum()
        lista_fiis = list(lista_fiis[lista_fiis>0].index)
    fii_dropdown.options = lista_fiis

In [5]:
# Importa dataset
df = pd.read_csv('FII_database.csv', encoding="ISO-8859-1")

# Converte coluna 'Data' para formato "datetime"
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

# Converte quantidade das vendas para negativo
df.loc[df['Operação']=='V','Quantidade'] *= -1

# Ajusta agrupamentos e desdobramentos
for fii in df[df['Operação']=='X']['Ativo'].unique(): #Varre FIIs com 'X'
    for data_X in list(df[(df['Ativo']==fii) & (df['Operação']=='X')]['Data']): #Recursivo em todos os 'X' de um mesmo FII
        fator = df[(df['Ativo']==fii) & (df['Operação']=='X') & (df['Data']==data_X)]['Quantidade'].values
        df.loc[(df['Ativo']==fii) & (df['Data']<data_X),'Quantidade'] *= fator
        df.loc[(df['Ativo']==fii) & (df['Data']<data_X),'Valor'] /= fator
        
# Cria coluna com o histórico do preço médio
df['Total'] = df['Quantidade'] * df['Valor'] + df['Taxas']  #Cria coluna com o valor total da operação
for fii in df[df['Operação'].isin(['C','V','SE'])]['Ativo'].unique():
    filtro = (df['Ativo']==fii) & (df['Operação'].isin(['C','V','SE']))
    df.loc[filtro,'Cotas'] = df[filtro]['Quantidade'].cumsum()  #Cria coluna com o saldo de cotas
    saldo = 0
    medio = []
    for quant,total,cotas in zip(df[filtro]['Quantidade'],df[filtro]['Total'],df[filtro]['Cotas']):
        if quant > 0:  #Compra
            saldo += total
            medio.append(saldo / cotas)
        elif cotas == 0:  #Zera saldo e PM se vender tudo
            saldo = 0
            medio.append(0)
        else:  #Venda
            saldo += quant * medio[-1]  #Soma "saldo" porque "quant" vem negativo nas vendas
            medio.append(medio[-1])     #Venda não altera PM (pega o último)
    df.loc[filtro,'PM'] = np.around(np.array(medio),2)

In [6]:
# Configura widgets
# Datas
data_ini_picker = widgets.DatePicker(value=df['Data'].min(), description='DATA INICIAL', style={'description_width':'82px'}, layout=widgets.Layout(width='230px'))
data_fim_picker = widgets.DatePicker(value=df['Data'].max(), description='DATA FINAL', style={'description_width':'82px'}, layout=widgets.Layout(width='230px'))
# Combo FIIs
fii_dropdown = widgets.Dropdown(layout=widgets.Layout(width='148px'))
carteira_checkbox = widgets.Checkbox(value=True, description='CARTEIRA ATUAL', indent=False, layout=widgets.Layout(width='95%', justify_content='center'))
atualiza_lista_fiis(carteira_checkbox) #Primeira carga da lista de FIIs
# Botão
botao_compras = widgets.Button(description='MOSTRAR DADOS', style=widgets.ButtonStyle(button_color='LightGray'), layout=widgets.Layout(height='95%'))
# Saída de resultados
output = widgets.Output()

# Chama função na alteração do checkbox
widgets.interactive(atualiza_lista_fiis, ck=carteira_checkbox)
# Chama função no click do botão
botao_compras.on_click(compras_clicked)

# Mostra widgets
vBox1 = widgets.VBox([data_ini_picker, data_fim_picker])
vBox2 = widgets.VBox([fii_dropdown, carteira_checkbox])
vBox3 = widgets.VBox([botao_compras])
menu = widgets.HBox([vBox1,vBox2,vBox3])
display(menu,output)

Output()